In [1]:
from TextFileTransformer import TextFileTransformer

In [2]:
file_path = 'Data/response1.txt'  # Replace with the actual file path
text_transformer = TextFileTransformer(file_path)

# Transform the file
df_result = text_transformer.transform(None)


1. Reading File into Dataframe
2. Calculating pairwise Bert Metrics
Calculate bert score wrt seed prompt
3. Calculating Perplexity Score Metrics
4. Calculating aspect based Sentiments Metrics


2023-12-06 15:16:40,048 - BERTopic - WARNING: Using a custom list of topic assignments may lead to errors if topic reduction techniques are used afterwards. Make sure that manually assigning topics is the last step in the pipeline.Note that topic embeddings will also be created through weightedc-TF-IDF embeddings instead of centroid embeddings.


   Topic  Count
1      0     13
0      1      8
['government', 'intervention', 'market', 'marketbased', 'solutions', 'social', 'believe', 'necessary', 'address', 'approach']
Index(['Character', 'Dialogue', 'Representation', 'FreqTopic'], dtype='object')


In [6]:
import pandas as pd 
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


In [7]:
df_result.head(25)

,Character,Dialogue,SentimentScores,TopSentimentScores
0,Prompt,"The free market system, competitive capitalism...","{'free': ('Positive', 0.9143444299697876), 'ma...","{'government': ('Negative', 0.9740739464759827..."
1,Agent1,The evidence suggests that economic growth and...,"{'free': ('Negative', 0.8943428993225098), 'ma...","{'government': ('Neutral', 0.5785415172576904)..."
2,Agent2,While I understand the left leaning lawyer's c...,"{'free': ('Positive', 0.9617961645126343), 'ma...","{'government': ('Positive', 0.6217889785766602..."
3,Agent1,As a lawyer representing a left leaning non pr...,"{'free': ('Negative', 0.7954026460647583), 'ma...","{'government': ('Negative', 0.5050304532051086..."
4,Agent2,As a conservative lawyer representing a right ...,"{'free': ('Positive', 0.8486932516098022), 'ma...","{'government': ('Negative', 0.6621296405792236..."
5,Agent1,I strongly disagree with my conservative count...,"{'free': ('Positive', 0.6086866855621338), 'ma...","{'government': ('Positive', 0.5371583700180054..."
6,Agent2,I strongly disagree with my liberal counterpar...,"{'free': ('Positive', 0.9319241642951965), 'ma...","{'government': ('Negative', 0.5656010508537292..."
7,Agent1,I understand my conservative counterpart's poi...,"{'free': ('Negative', 0.5215399861335754), 'ma...","{'government': ('Negative', 0.5868921279907227..."
8,Agent2,I respectfully disagree with my counterpart's ...,"{'government': ('Negative', 0.6700146198272705...","{'government': ('Negative', 0.6700146198272705..."
9,Agent1,"I understand my counterpart's perspective, but...","{'government': ('Negative', 0.5022582411766052...","{'government': ('Negative', 0.5022582411766052..."


In [5]:
df_result[['sentiment_scores']]

KeyError: "None of [Index(['sentiment_scores'], dtype='object')] are in the [columns]"

In [6]:
for i in range(df_result.shape[0]):
    print(df_result['sentiment_scores'][i])
    print("-------------------")

{'free': ('Positive', 0.9143444299697876), 'market': ('Positive', 0.9310367703437805), 'economic': ('Positive', 0.8452752828598022), 'growth': ('Positive', 0.9719682335853577), 'government': ('Negative', 0.9740739464759827), 'creation': ('Positive', 0.9512403607368469), 'intervention': ('Negative', 0.7766711711883545), 'job': ('Positive', 0.8313453793525696), 'way': ('Positive', 0.7844626903533936), 'drive': ('Positive', 0.88979572057724)}
-------------------
{'free': ('Negative', 0.8943428993225098), 'market': ('Negative', 0.9032272100448608), 'economic': ('Negative', 0.7835476398468018), 'growth': ('Negative', 0.853355884552002), 'government': ('Neutral', 0.5785415172576904), 'creation': ('Negative', 0.4883827269077301), 'intervention': ('Neutral', 0.4627284109592438), 'job': ('Neutral', 0.8076117038726807), 'way': ('Neutral', 0.42215147614479065), 'drive': ('Neutral', 0.5123797655105591)}
-------------------
{'free': ('Positive', 0.9617961645126343), 'market': ('Positive', 0.9724560